<a href="https://colab.research.google.com/github/prometheus404/AMD_project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##### DOWNLOAD DATASET #####
from IPython.display import clear_output
from google.colab import files
files.upload()
!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d gsimonx37/letterboxd
!unzip /content/letterboxd.zip -d /content/letterbox/
clear_output()
############################

In [2]:
########### SPARK CONTEXT #####################
import pandas as pd
import itertools
from tqdm import tqdm

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

sc = spark.sparkContext
###############################################

In [ ]:
############## BASKET CREATION ################
#TODO do it using SPARK directly -> cars = spark.read.csv('cars.csv', header=True, sep=";")
actors = pd.read_csv("letterbox/actors.csv")
actors = actors
baskets_full = actors.groupby("id")["name"].apply(list)
baskets = baskets_full.sample(30000, random_state = 42)

print("number of baskets: " + str(len(baskets)))
print("biggest basket: " + str(baskets.map(len).max()))
print(baskets)
baskets_RDD = sc.parallelize(baskets).cache()
##############################################

number of baskets: 30000
biggest basket: 246
id
1061722    [Victor Andres Trelles Turgeon, Paul-Antoine T...
1780554    [Günter Pfitzmann, Gerd Duwner, Inge Wolffberg...
1049826    [Andressa Koetz, Juninho Bill, Paulo Reis, Zac...
1113595    [Kurt Sinclair, Leslie Taylor, Kathleen Klein,...
1594379    [Çetin Başaran, Naki Yurter, Recep Filiz, Haka...
                                 ...                        
1483472    [Song Yoo-dam, Haechan, Kim Sang-gyun, Kim Si-...
1374678    [Taeko Yoshida, Mitsuru Tamaki, Masao Teruya, ...
1304228    [Megumi Okada, Kikuko Inoue, Aya Endo, Kanehir...
1749248                                      [Chloë Fratani]
1875572                                        [Nick Cheung]
Name: name, Length: 30000, dtype: object


In [53]:
## APRIORI
def apriori(chunk, s, tot_bsk):

  n_bsk = 0 # number of baskets in chunk used to calculate support
#  trsl = dict() # using a translation could be better for the combination step
  count = dict() # should be relatively efficient since is a hash table

#
#  def translate(e, dic):
#    try:
#      return dic[e]
#    except KeyError:
#      dic[e] = len(dic)+1
#      return dic[e]
  def incr(e,dic):
    try:
      dic[e] += 1;
    except KeyError:
      dic[e] = 1

  chunk = list(chunk) # to avoid consuming after first pass
  ### first pass
  for bsk in chunk:
    n_bsk += 1;
    #elements = [translate(x,trsl) for x in bsk]
    for b in bsk:
      incr(b,count)

  ### filter
  to_del = []
  frequent = []
  for c in count:
    if count[c] >= s* n_bsk/tot_bsk:
      frequent.append(c)

  k = 2
  while len(frequent) > 0:
    count = dict()

    for bsk in tqdm(chunk):
      #filter only members of C_k
      filtered = set()
      for b in bsk:
        for f in frequent:
          if b in f:
            filtered.add(b)
            break;
      bsk = list(filtered)
      bsk.sort()
      for tpl in itertools.combinations(bsk, k):
          incr(tuple(tpl), count)

    frequent = []
    for c in count:
      if count[c] >= s * n_bsk/tot_bsk:
        frequent.append(c)
        yield(c,1)
    k+=1


In [56]:
def son(rdd,s,tot):
  def chunk_support(chunk,itemset):
    def incr(e,dic):
      try:
        dic[e] += 1;
      except KeyError:
        dic[e] = 1

    count = dict()
    for bsk in tqdm(chunk):
      for i in itemset:
        flag = True
        for e in i:
          if e not in bsk:
            flag = False
            break
        if flag:
          incr(i,count)
    for c in count:
      yield (c,count[c])
  #FIRST STEP
  candidates = rdd.mapPartitions(lambda x: apriori(x,s,tot)).reduceByKey(lambda a,b: a).keys().collect()
  #SECOND STEP
  return rdd.mapPartitions(lambda x: chunk_support(x, candidates)).reduceByKey(lambda a,b: a+b).filter(lambda a: a[1] >= s).keys().collect()

In [ ]:
son(baskets_RDD, 6, len(baskets))

[('Jeon Jung-kook', 'Jung Ho-seok'),
 ('Jeon Jung-kook', 'Kim Seok-jin'),
 ('Jeon Jung-kook', 'Kim Tae-hyung'),
 ('Jeon Jung-kook', 'Min Yoon-gi'),
 ('Jung Ho-seok', 'Kim Seok-jin'),
 ('Jung Ho-seok', 'Kim Tae-hyung'),
 ('Jung Ho-seok', 'Min Yoon-gi'),
 ('Kim Nam-joon', 'Park Ji-min'),
 ('Kim Seok-jin', 'Kim Tae-hyung'),
 ('Kim Seok-jin', 'Min Yoon-gi'),
 ('Kim Tae-hyung', 'Min Yoon-gi'),
 ('Jeon Jung-kook', 'Jung Ho-seok', 'Kim Seok-jin'),
 ('Jeon Jung-kook', 'Jung Ho-seok', 'Kim Tae-hyung'),
 ('Jeon Jung-kook', 'Jung Ho-seok', 'Min Yoon-gi'),
 ('Jeon Jung-kook', 'Kim Nam-joon', 'Park Ji-min'),
 ('Jeon Jung-kook', 'Kim Seok-jin', 'Kim Tae-hyung'),
 ('Jeon Jung-kook', 'Kim Seok-jin', 'Min Yoon-gi'),
 ('Jeon Jung-kook', 'Kim Tae-hyung', 'Min Yoon-gi'),
 ('Jung Ho-seok', 'Kim Nam-joon', 'Park Ji-min'),
 ('Jung Ho-seok', 'Kim Seok-jin', 'Kim Tae-hyung'),
 ('Jung Ho-seok', 'Kim Seok-jin', 'Min Yoon-gi'),
 ('Jung Ho-seok', 'Kim Tae-hyung', 'Min Yoon-gi'),
 ('Kim Nam-joon', 'Kim Seok-jin', '

In [ ]:
execute_full = True
result = []
if execute_full:
  print("cc")
  full_RDD = sc.parallelize(baskets_full).cache()
  for i in apriori(baskets_full, 140, len(baskets_full)):
    result.append(i[0])
print(result)

cc


100%|██████████| 603163/603163 [00:02<00:00, 269889.57it/s]

[('Larry Fine', 'Moe Howard'), ('Oliver Hardy', 'Stan Laurel'), ('James Hetfield', 'Kirk Hammett'), ('James Hetfield', 'Lars Ulrich'), ('Kirk Hammett', 'Lars Ulrich'), ('Jack Mercer', 'Mae Questel'), ('Ali Basha', 'Brahmanandam'), ('Bebe Daniels', 'Harold Lloyd'), ('Bebe Daniels', "Harry 'Snub' Pollard"), ('Harold Lloyd', "Harry 'Snub' Pollard"), ('James Hetfield', 'Kirk Hammett', 'Lars Ulrich'), ('Bebe Daniels', 'Harold Lloyd', "Harry 'Snub' Pollard")]


In [ ]:
###############################################################

603163

In [9]:
tmp2 = sc.parallelize(range(0,1000))
print(tmp2.sample(False,fraction=0.001, seed = 42).collect())
print(tmp2.sample(False,fraction=0.002, seed = 42).collect())
print(tmp2.sample(False,fraction=0.003, seed = 42).collect())
# NOTE it's important to change seed even with different fractions

[115, 385]
[115, 359, 385]
[115, 359, 385]


In [35]:
RDD = sc.textFile("letterbox/actors.csv").map(lambda x: tuple(x.split(','))) #read from csv and split
RDD = RDD.filter(lambda x: x != ('id','name')) #filter out the header
#construct dictionary for translation
RDD_dict = RDD.sortBy(lambda x: x[1]).values().distinct().zipWithIndex() # not zipWithUniquenId since we could need gap-less id
RDD_dict.collect()

[('"""Bowpicker"" Bob Merel"', 0),
 ('"""CJ Chris James"', 1),
 ('"""Daddy"" Gene Jackson"', 2),
 ('"""Damegane"""', 3),
 ('"""El Bene"" Ramos"', 4),
 ('"""Goldman"" Takeuchi"', 5),
 ('"""Hollywood Jade"" Anderson"', 6),
 ('"""Lil"" Sam Herman"', 7),
 ('"""Marukichi"""', 8),
 ('"""Milkshake No"" Woman"', 9),
 ('"""Puton"""', 10),
 ('"""Spartain Mouse"""', 11),
 ('"""Torch"" Findley"', 12),
 ('"""Xander"" Truelsen"', 13),
 ('"Abner Delina', 14),
 ('"Ada ""Bricktop"" Smith"', 15),
 ('"Agrecio C. Basul', 16),
 ('"Alfred Sauchelli', 17),
 ('"Allan Skeene', 18),
 ('"Alondra Ra', 19),
 ('"Alpha Fowler', 20),
 ('"Alphaeus Green', 21),
 ('"Alphonse ""Bois Sec"" Ardoin"', 22),
 ('"Alvar ""Masen"" Eriksson"', 23),
 ('"Ambrus Deák ""AMB"""', 24),
 ('"Amir ""Bink"" Johnson"', 25),
 ('"Ana ""Frango Elétrico"""', 26),
 ('"Ana Đurić ""Konstrakta"""', 27),
 ('"Anders ""Blakkheim"" Nyström"', 28),
 ('"Andrea ""Andy"" Ferro"', 29),
 ('"Andrew Newberg', 30),
 ('"Andriy ""Fedot"" Fedotov"', 31),
 ('"Andrá

In [36]:
RDD = RDD.map(lambda x: (x[1],x[0])) #invert keys with values in order to perform the join over the actor name
RDD = RDD.join(RDD_dict).map(lambda x: (x[1][0],x[1][1])) #join and use as key the movie and as value the actor id
RDD = RDD.groupByKey().mapValues(list).values().cache() #create the baskets and cache them

In [51]:
for i in tqdm(range(10000, 60000, 10000)):
  sampled_RDD = RDD.sample(False, i/603163, seed = int(i/10000)) #fixed but different for each iteration seed
  sampled_RDD.saveAsTextFile("./rdd-"+str(i)+".csv")
  #son(sampled_RDD,6,sampled_RDD.count())

100%|██████████| 5/5 [00:08<00:00,  1.78s/it]


In [45]:
RDD.count()

603163

In [55]:
x = sc.textFile("rdd-10000.csv/")
son(x,6,10000)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 101.0 failed 1 times, most recent failure: Lost task 0.0 in stage 101.0 (TID 302) (3a26669a52bb executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/content/spark-3.1.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/content/spark-3.1.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/content/spark-3.1.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/content/spark-3.1.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-54-490c00a73d25>", line 

In [65]:
x = x.map(lambda s: s[1:-1].split(',')).map(lambda s: [int(i) for i in s]).collect()

In [66]:
son(x,6,10000)

AttributeError: 'list' object has no attribute 'mapPartitions'